# Fetch Rewards - Assessment

In [87]:
import pandas as pd
import json
from datetime import datetime

# Loading the json gzipped file into a dataframe

In [2]:
#loading the json gzipped file into a dataframe
receipts = pd.read_json(r"D:\receipts.json.gz", compression="gzip", lines=True)

In [3]:
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


**Converting all the Date columns to %Y-%m-%d %H:%M:%S format, making it more readable**

In [4]:
def date_converter(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)


**Extracting values from json columns and getting the date in right format**

In [7]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_converter(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_converter(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_converter(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_converter(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_converter(x))

## Total Spent is missing for 435 rows of data

In [9]:
receipts["totalSpent"].isna().sum()

435

In [24]:
receipts["pointsEarned"].isna().sum()

510

## Over 500 pointsEarned values are malformed or null - this could break analytics unless cleaned.

In [72]:
malformed_values = receipts["pointsEarned"][~receipts["pointsEarned"].astype(str).str.replace('.', '', 1).str.isnumeric()]
print(f"Malformed 'pointsEarned' values: {len(malformed_values)}")

Malformed 'pointsEarned' values: 510


## All the id in the receipt dataframe are unique

In [30]:
receipts['_id'].nunique()

1119

## purchaseDate column are formatted, but they still have many null values

In [32]:
receipts.purchaseDate.isna().sum()

448

# Receipts Data Findings:

- Nearly 40% of receipts are missing spend/purchase/item info, indicating potentially incomplete ingestion or partial scans.
- Over 500 pointsEarned values are malformed (non-numeric or strings like "N/A", etc.) — this could break analytics unless cleaned.
- Date columns are present and now consistently formatted, but many are null.

-----------------------------------------------------------------------------------

In [38]:
receipts['rewardsReceiptItemList'].str.len().sum()

6941.0

## rewardsReceiptItemList has a lenght of 6941. Exploding the column into a separate 'items_df' dataframe, we get:

In [35]:
items_data = []

for idx, row in receipts.iterrows():
    items = row["rewardsReceiptItemList"]
    if isinstance(items, list):
        for item in items:
            item["receipt_id"] = row["_id"]
            item["userId"] = row["userId"]
            items_data.append(item)

# Convert the list of dictionaries into a DataFrame
items_df = pd.DataFrame(items_data)

In [37]:
items_df

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [90]:
items_df.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'receipt_id', 'userId', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId', 'quantityPurchased_str', 'itemPrice_str',
       'finalPrice_str', 'quantityPurchased_num', 'itemPrice_num',
       'finalPrice_num'],
      dtype='object')

## Nearly 50% of the barcodes in the dataframe are null

In [39]:
items_df.barcode.isna().sum()

3851

## Missing Description: Product label not captured, which limits usability

In [91]:
items_df.description.isna().sum()

381

In [41]:
items_df.quantityPurchased.value_counts()

1.0     5628
2.0      622
4.0      170
3.0      134
5.0      101
6.0       37
8.0       22
10.0      15
9.0       13
7.0       13
12.0       6
17.0       3
14.0       3
Name: quantityPurchased, dtype: int64

# Loading the Brands Data

In [52]:
brands = pd.read_json(r"D:\brands.json.gz", compression="gzip", lines=True)

In [65]:
brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


## cpg column, which is a key value pair into 2 different columns

In [53]:
brands_json = pd.json_normalize(brands['cpg'])

In [54]:
brands_json

,$ref,$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


#### $ref value counts below - 'Cogs' make up more than 90%

In [62]:
brands_json['$ref'].value_counts()

Cogs    1020
Cpgs     147
Name: $ref, dtype: int64

In [68]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])
brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


### All the brand ids are unique

In [71]:
brands._id.nunique()

1167

## Brand Code has lot of rows which has null value

In [63]:
brands['brandCode'].value_counts()

                                   35
HUGGIES                             2
GOODNITES                           2
KELSEN                              1
THE GLENLIVET CARIBBEAN RESERVE     1
                                   ..
TEST BRANDCODE @1598711015496       1
ABSOLUT® MANDRIN                    1
TEST BRANDCODE @1598711015353       1
TEST BRANDCODE @1610660035741       1
F WHITLOCK AND SONS                 1
Name: brandCode, Length: 897, dtype: int64

In [64]:
brands['brandCode'].isna().sum()

234

### Around 10% are missing category values, which may impact category-based reporting.

In [67]:
brands.category.isna().sum()

155

In [94]:
brands[brands['topBrand'] == 1.0]

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
58,5c76d3cd95144c5375687b4f,511111106876,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5c76d2059514...",DASH-2249 Brand1,1.0,TEST BRAND CODE
109,585a9645e4b03e62d1ce0e79,511111801757,Snacks,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Chester's,1.0,CHESTER'S
115,5887a372e4b02187f85cdad9,511111001119,Snacks,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Doritos,1.0,DORITOS
116,57ed0697e4b072ac2294b8f2,511111101895,Condiments & Sauces,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",A.1.,1.0,A.1.
152,5c45f91b87ff3552f950f027,511111204923,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5c45f8b087ff...",Brand1,1.0,0987654321
192,585a963ce4b03e62d1ce0e78,511111001768,Snacks,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Cheetos,1.0,CHEETOS
245,5d66961cee7f2d201c7281cc,511111812449,Magazines,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5c4f20b01b41...",Test brand1,1.0,NaN
258,57c0827de4b0718ff5fcb037,511111902461,Baby,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f7a7e4b0...",Antarctica,1.0,AMP2
271,585a9637e4b03e62d1ce0e77,511111501770,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Cap'n Crunch,1.0,CAP'N CRUNCH
278,585a967fe4b03e62d1ce0e80,511111801689,Snacks,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Lay's Kettle Cooked,1.0,


## Unique number of brand code differ in the items and brands dataframe

In [92]:
items_df.brandCode.nunique()

227

In [93]:
brands.brandCode.nunique()

897

# Brand data findings:
- Brand table is relatively clean — all records have valid IDs and names.
- About 20% are missing brand codes, which may impact brand-related analysis
- About 13% are missing category values, which may impact category-based reporting.


# Loading the Users Data frame

In [77]:
users = pd.read_json(r"D:\users.json",lines=True)
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


## Cleaning the json _id columns, and the date columns

In [78]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [79]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN


## High number of duplicate user records (over 50% are duplicates).

In [80]:
users._id.nunique()

212

## There is a discrepancy in the count of userId
- Unique userId in the receipts dataframe : 258
- Unique Id in the users dataframe : 212

## This may be due to missing userIds in the Users dataframe

In [86]:
receipts.userId.nunique()

258

## About 10% of the states listed are null

In [83]:
users.state.isna().sum()

56

## 62 lastLogin values are null, which will affect analysis on when the user was last using the account

In [85]:
users.lastLogin.isna().sum()

62

# Users Findings:
- High number of duplicate user records (over 50% are duplicates).
- Some users have invalid or missing state entries — possibly dirty manual entries or system errors.
- lastLogin is missing for several users — may indicate inactive or incomplete accounts.